In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.9 MB/s eta 0:00:00


In [5]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import torch
from transformers import GPT2Tokenizer, GPT2Model

# Mount Google Drive
drive.mount('/content/drive')

# Function to extract GPT-2 embeddings for the text column
def extract_gpt2_embeddings(input_file, text_column):
    # Load dataset
    df = pd.read_excel(input_file)

    # Load GPT-2 model and tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    model = GPT2Model.from_pretrained("gpt2")

    # Add a padding token to the tokenizer if it does not have one
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token  # Use eos_token as pad_token

    # Function to convert text to embeddings
    def get_embeddings(text):
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()  # Get the mean of token embeddings
        return embeddings.numpy()

    # Extract embeddings for all text
    embeddings = np.array([get_embeddings(text) for text in df[text_column]])

    # Convert embeddings into a DataFrame
    embeddings_df = pd.DataFrame(embeddings)

    # Return embeddings dataframe and original data
    return embeddings_df, df

# Function to apply PCA with automatic component selection based on explained variance
def apply_pca(X, variance_threshold=0.95):
    from sklearn.decomposition import PCA
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Model functions (same as your previous code)
# Model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")


def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40],
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 2, 4]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)


# Input file and feature details
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'answer'
numeric_features = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']

# Extract GPT-2 embeddings
X, original_data = extract_gpt2_embeddings(input_file, text_column)

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(X, variance_threshold=0.95)  # Adjust variance threshold as needed

# Prepare target variable
y = original_data['confidence'].apply(lambda x: x + 1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of components selected: 2
Running KNN model...


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Params: {'weights': 'distance', 'n_neighbors': 5}
KNN Model Metrics:
Train Accuracy: 0.9958
Train Precision: 0.9958
Train Recall: 0.9958
Train F1 Score: 0.9958
Train Confusion Matrix:
[[ 32   0   0]
 [  0 130   2]
 [  0   0 312]]

Test Accuracy: 0.6390
Test Precision: 0.6335
Test Recall: 0.6390
Test F1 Score: 0.6308
Test Confusion Matrix:
[[  1   5   7]
 [  1  11  29]
 [  1  31 119]]

Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 5, 'max_depth': 20}
Random Forest Model Metrics:
Train Accuracy: 0.9391
Train Precision: 0.9420
Train Recall: 0.9391
Train F1 Score: 0.9332
Train Confusion Matrix:
[[ 15   4  13]
 [  0 121  11]
 [  0   1 311]]

Test Accuracy: 0.6488
Test Precision: 0.6025
Test Recall: 0.6488
Test F1 Score: 0.6204
Test Confusion Matrix:
[[  1   2  10]
 [  2   6  33]
 [  1  24 126]]

Running Decision Tree model...
Best Params: {'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 40}
Decision Tree Model Metrics:
Train Accuracy

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'sgd', 'hidden_layer_sizes': (50,), 'activation': 'tanh'}
MLP Model Metrics:
Train Accuracy: 0.6576
Train Precision: 0.5978
Train Recall: 0.6576
Train F1 Score: 0.5312
Train Confusion Matrix:
[[  0   0  32]
 [  0   3 129]
 [  0   2 310]]

Test Accuracy: 0.7268
Test Precision: 0.5433
Test Recall: 0.7268
Test F1 Score: 0.6218
Test Confusion Matrix:
[[  0   1  12]
 [  0   0  41]
 [  0   2 149]]

Running XGBoost model...
Best Params: {'subsample': 1.0, 'n_estimators': 50, 'learning_rate': 0.1}
XGBoost Model Metrics:
Train Accuracy: 0.8361
Train Precision: 0.8571
Train Recall: 0.8361
Train F1 Score: 0.8206
Train Confusion Matrix:
[[ 16   3  13]
 [  0  72  60]
 [  0   2 310]]

Test Accuracy: 0.6683
Test Precision: 0.6102
Test Recall: 0.6683
Test F1 Score: 0.6336
Test Confusion Matrix:
[[  1   2  10]
 [  2   6  33]
 [  3  18 130]]

Running AdaBoost model...


...

Best Params: {'learning_rate': 0.3, 'iterations': 150, 'depth': 6}
CatBoost Model Metrics:
Train Accuracy: 0.8929
Train Precision: 0.9023
Train Recall: 0.8929
Train F1 Score: 0.8844
Train Confusion Matrix:
[[ 14   4  14]
 [  0 100  32]
 [  0   1 311]]

Test Accuracy: 0.6390
Test Precision: 0.6072
Test Recall: 0.6390
Test F1 Score: 0.6205
Test Confusion Matrix:
[[  1   2  10]
 [  2   8  31]
 [  2  27 122]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.01}
Logistic Regression Model Metrics:
Train Accuracy: 0.6366
Train Precision: 0.5339
Train Recall: 0.6366
Train F1 Score: 0.5569
Train Confusion Matrix:
[[  0   5  27]
 [  0  16 116]
 [  0  25 287]]

Test Accuracy: 0.7220
Test Precision: 0.6229
Test Recall: 0.7220
Test F1 Score: 0.6523
Test Confusion Matrix:
[[  0   1  12]
 [  0   5  36]
 [  0   8 143]]

Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.6618
Train Precision: 0.6186
Train Recall: 0.6618
Train F1 Score: 0.5374
Train Confusion Matrix:
[[  0   1  31]
 [  0   4 128]
 [  0   1 311]]

Test Accuracy: 0.7317
Test Precision: 0.6100
Test Recall: 0.7317
Test F1 Score: 0.6309
Test Confusion Matrix:
[[  0   0  13]
 [  0   1  40]
 [  0   2 149]]



In [13]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import torch
from transformers import GPT2Tokenizer, GPT2Model

# Mount Google Drive
drive.mount('/content/drive')

# Function to extract GPT-2 embeddings for the text column
def extract_gpt2_embeddings(input_file, text_column):
    # Load dataset
    df = pd.read_excel(input_file)

    # Load GPT-2 model and tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    model = GPT2Model.from_pretrained("gpt2")

    # Add a padding token to the tokenizer if it does not have one
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token  # Use eos_token as pad_token

    # Function to convert text to embeddings
    def get_embeddings(text):
        # Ensure text is a string
        text = str(text)  # Convert to string if not already

        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()  # Get the mean of token embeddings
        return embeddings.numpy()

    # Extract embeddings for all text
    embeddings = np.array([get_embeddings(text) for text in df[text_column]])

    # Convert embeddings into a DataFrame
    embeddings_df = pd.DataFrame(embeddings)

    # Return embeddings dataframe and original data
    return embeddings_df, df

# Function to apply PCA with automatic component selection based on explained variance
def apply_pca(X, variance_threshold=0.95):
    from sklearn.decomposition import PCA
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Model functions (same as your previous code)
# Model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")


def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40],
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 2, 4]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)


# Input file and feature details
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'cleaned_answer'
numeric_features = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']

# Extract GPT-2 embeddings
X, original_data = extract_gpt2_embeddings(input_file, text_column)

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(X, variance_threshold=0.95)  # Adjust variance threshold as needed

# Prepare target variable
y = original_data['confidence'].apply(lambda x: x + 1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of components selected: 2
Running KNN model...
Best Params: {'weights': 'uniform', 'n_neighbors': 7}
KNN Model Metrics:
Train Accuracy: 0.7059
Train Precision: 0.6709
Train Recall: 0.7059
Train F1 Score: 0.6654
Train Confusion Matrix:
[[  2   9  21]
 [  2  47  83]
 [  1  24 287]]

Test Accuracy: 0.6146
Test Precision: 0.5625
Test Recall: 0.6146
Test F1 Score: 0.5873
Test Confusion Matrix:
[[  0   4   9]
 [  0   5  36]
 [  0  30 121]]

Running Random Forest model...
Best Params: {'n_estimators': 150, 'min_samples_split': 10, 'max_depth': 10}
Random Forest Model Metrics:
Train Accuracy: 0.7878
Train Precision: 0.8114
Train Recall: 0.7878
Train F1 Score: 0.7509
Train Confusion Matrix:
[[  3   2  27]
 [  1  62  69]
 [  0   2 310]]

Test Accuracy: 0.6927
Test Precision: 0.6444
Test Recall: 0.6927
Test F1 Score: 0.6341
Test Confusion Matrix:
[[  1   3   9]
 

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (50,), 'activation': 'tanh'}
MLP Model Metrics:
Train Accuracy: 0.6723
Train Precision: 0.6024
Train Recall: 0.6723
Train F1 Score: 0.5822
Train Confusion Matrix:
[[  0   4  28]
 [  0  17 115]
 [  0   9 303]]

Test Accuracy: 0.7024
Test Precision: 0.5753
Test Recall: 0.7024
Test F1 Score: 0.6232
Test Confusion Matrix:
[[  0   1  12]
 [  0   2  39]
 [  0   9 142]]

Running XGBoost model...
Best Params: {'subsample': 0.5, 'n_estimators': 100, 'learning_rate': 0.1}
XGBoost Model Metrics:
Train Accuracy: 0.9013
Train Precision: 0.9079
Train Recall: 0.9013
Train F1 Score: 0.8955
Train Confusion Matrix:
[[ 16   1  15]
 [  0 106  26]
 [  0   5 307]]

Test Accuracy: 0.6439
Test Precision: 0.6732
Test Recall: 0.6439
Test F1 Score: 0.6330
Test Confusion Matrix:
[[  1   5   7]
 [  0  11  30]
 [  0  31 120]]

Running AdaBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=30. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'n_estimators': 150, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.6597
Train Precision: 0.7088
Train Recall: 0.6597
Train F1 Score: 0.5286
Train Confusion Matrix:
[[  0   0  32]
 [  0   2 130]
 [  0   0 312]]

Test Accuracy: 0.7366
Test Precision: 0.5426
Test Recall: 0.7366
Test F1 Score: 0.6249
Test Confusion Matrix:
[[  0   0  13]
 [  0   0  41]
 [  0   0 151]]

Running CatBoost model...


...

Best Params: {'learning_rate': 0.1, 'iterations': 100, 'depth': 6}
CatBoost Model Metrics:
Train Accuracy: 0.7248
Train Precision: 0.7452
Train Recall: 0.7248
Train F1 Score: 0.6653
Train Confusion Matrix:
[[  3   3  26]
 [  1  35  96]
 [  0   5 307]]

Test Accuracy: 0.7024
Test Precision: 0.6590
Test Recall: 0.7024
Test F1 Score: 0.6452
Test Confusion Matrix:
[[  1   3   9]
 [  0   4  37]
 [  0  12 139]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.01}
Logistic Regression Model Metrics:
Train Accuracy: 0.6555
Train Precision: 0.5413
Train Recall: 0.6555
Train F1 Score: 0.5384
Train Confusion Matrix:
[[  0   3  29]
 [  0   5 127]
 [  0   5 307]]

Test Accuracy: 0.7317
Test Precision: 0.6145
Test Recall: 0.7317
Test F1 Score: 0.6400
Test Confusion Matrix:
[[  0   1  12]
 [  0   2  39]
 [  0   3 148]]

Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'linear', 'C': 0.1}
SVM Model Metrics:
Train Accuracy: 0.6555
Train Precision: 0.4296
Train Recall: 0.6555
Train F1 Score: 0.5190
Train Confusion Matrix:
[[  0   0  32]
 [  0   0 132]
 [  0   0 312]]

Test Accuracy: 0.7366
Test Precision: 0.5426
Test Recall: 0.7366
Test F1 Score: 0.6249
Test Confusion Matrix:
[[  0   0  13]
 [  0   0  41]
 [  0   0 151]]



In [15]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import torch
from transformers import GPT2Tokenizer, GPT2Model

# Mount Google Drive
drive.mount('/content/drive')

# Function to extract GPT-2 embeddings for the text column
def extract_gpt2_embeddings(input_file, text_column):
    # Load dataset
    df = pd.read_excel(input_file)

    # Load GPT-2 model and tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    model = GPT2Model.from_pretrained("gpt2")

    # Add a padding token to the tokenizer if it does not have one
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token  # Use eos_token as pad_token

    # Function to convert text to embeddings
    def get_embeddings(text):
        # Ensure text is a string
        text = str(text)  # Convert to string if not already
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()  # Get the mean of token embeddings
        return embeddings.numpy()

    # Extract embeddings for all text
    embeddings = np.array([get_embeddings(text) for text in df[text_column]])

    # Convert embeddings into a DataFrame
    embeddings_df = pd.DataFrame(embeddings)

    # Return embeddings dataframe and original data
    return embeddings_df, df

# Function to apply PCA with automatic component selection based on explained variance
def apply_pca(X, variance_threshold=0.95):
    from sklearn.decomposition import PCA
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Model functions (same as your previous code)
# Model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")


def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40],
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 2, 4]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)


# Input file and feature details
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'cleaned_answer'


# Extract GPT-2 embeddings
X, original_data = extract_gpt2_embeddings(input_file, text_column)

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(X, variance_threshold=0.95)  # Adjust variance threshold as needed

# Prepare target variable
y = original_data['confidence'].apply(lambda x: x + 1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of components selected: 2
Running KNN model...
Best Params: {'weights': 'uniform', 'n_neighbors': 7}
KNN Model Metrics:
Train Accuracy: 0.7059
Train Precision: 0.6709
Train Recall: 0.7059
Train F1 Score: 0.6654
Train Confusion Matrix:
[[  2   9  21]
 [  2  47  83]
 [  1  24 287]]

Test Accuracy: 0.6146
Test Precision: 0.5625
Test Recall: 0.6146
Test F1 Score: 0.5873
Test Confusion Matrix:
[[  0   4   9]
 [  0   5  36]
 [  0  30 121]]

Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 5, 'max_depth': 30}
Random Forest Model Metrics:
Train Accuracy: 0.9244
Train Precision: 0.9267
Train Recall: 0.9244
Train F1 Score: 0.9204
Train Confusion Matrix:
[[ 18   2  12]
 [  1 113  18]
 [  0   3 309]]

Test Accuracy: 0.6780
Test Precision: 0.6737
Test Recall: 0.6780
Test F1 Score: 0.6483
Test Confusion Matrix:
[[  1   5   7]
 [ 

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (100,), 'activation': 'tanh'}
MLP Model Metrics:
Train Accuracy: 0.6702
Train Precision: 0.6007
Train Recall: 0.6702
Train F1 Score: 0.5682
Train Confusion Matrix:
[[  0   4  28]
 [  0  12 120]
 [  0   5 307]]

Test Accuracy: 0.7268
Test Precision: 0.6040
Test Recall: 0.7268
Test F1 Score: 0.6372
Test Confusion Matrix:
[[  0   1  12]
 [  0   2  39]
 [  0   4 147]]

Running XGBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'subsample': 0.5, 'n_estimators': 150, 'learning_rate': 0.1}
XGBoost Model Metrics:
Train Accuracy: 0.9580
Train Precision: 0.9591
Train Recall: 0.9580
Train F1 Score: 0.9573
Train Confusion Matrix:
[[ 27   1   4]
 [  0 119  13]
 [  0   2 310]]

Test Accuracy: 0.6439
Test Precision: 0.6084
Test Recall: 0.6439
Test F1 Score: 0.6256
Test Confusion Matrix:
[[  0   5   8]
 [  0  11  30]
 [  0  30 121]]

Running AdaBoost model...


...

Best Params: {'learning_rate': 0.3, 'iterations': 50, 'depth': 6}
CatBoost Model Metrics:
Train Accuracy: 0.7605
Train Precision: 0.7794
Train Recall: 0.7605
Train F1 Score: 0.7183
Train Confusion Matrix:
[[  3   2  27]
 [  1  53  78]
 [  0   6 306]]

Test Accuracy: 0.6878
Test Precision: 0.5951
Test Recall: 0.6878
Test F1 Score: 0.6342
Test Confusion Matrix:
[[  0   4   9]
 [  0   5  36]
 [  0  15 136]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.01}
Logistic Regression Model Metrics:
Train Accuracy: 0.6555
Train Precision: 0.5413
Train Recall: 0.6555
Train F1 Score: 0.5384
Train Confusion Matrix:
[[  0   3  29]
 [  0   5 127]
 [  0   5 307]]

Test Accuracy: 0.7317
Test Precision: 0.6145
Test Recall: 0.7317
Test F1 Score: 0.6400
Test Confusion Matrix:
[[  0   1  12]
 [  0   2  39]
 [  0   3 148]]

Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'linear', 'C': 0.1}
SVM Model Metrics:
Train Accuracy: 0.6555
Train Precision: 0.4296
Train Recall: 0.6555
Train F1 Score: 0.5190
Train Confusion Matrix:
[[  0   0  32]
 [  0   0 132]
 [  0   0 312]]

Test Accuracy: 0.7366
Test Precision: 0.5426
Test Recall: 0.7366
Test F1 Score: 0.6249
Test Confusion Matrix:
[[  0   0  13]
 [  0   0  41]
 [  0   0 151]]



In [17]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import torch
from transformers import GPT2Tokenizer, GPT2Model

# Mount Google Drive
drive.mount('/content/drive')

# Function to extract GPT-2 embeddings for the text column
def extract_gpt2_embeddings(input_file, text_column):
    # Load dataset
    df = pd.read_excel(input_file)

    # Load GPT-2 model and tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    model = GPT2Model.from_pretrained("gpt2")

    # Add a padding token to the tokenizer if it does not have one
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token  # Use eos_token as pad_token

    # Function to convert text to embeddings
    def get_embeddings(text):
        # Ensure text is a string
        text = str(text)  # Convert to string if not already
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()  # Get the mean of token embeddings
        return embeddings.numpy()

    # Extract embeddings for all text
    embeddings = np.array([get_embeddings(text) for text in df[text_column]])

    # Convert embeddings into a DataFrame
    embeddings_df = pd.DataFrame(embeddings)

    # Return embeddings dataframe and original data
    return embeddings_df, df

# Function to apply PCA with automatic component selection based on explained variance
def apply_pca(X, variance_threshold=0.95):
    from sklearn.decomposition import PCA
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Model functions (same as your previous code)
# Model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")


def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40],
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 2, 4]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)


# Input file and feature details
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'answer'


# Extract GPT-2 embeddings
X, original_data = extract_gpt2_embeddings(input_file, text_column)

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(X, variance_threshold=0.95)  # Adjust variance threshold as needed

# Prepare target variable
y = original_data['confidence'].apply(lambda x: x + 1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of components selected: 2
Running KNN model...
Best Params: {'weights': 'distance', 'n_neighbors': 5}
KNN Model Metrics:
Train Accuracy: 0.9958
Train Precision: 0.9958
Train Recall: 0.9958
Train F1 Score: 0.9958
Train Confusion Matrix:
[[ 32   0   0]
 [  0 130   2]
 [  0   0 312]]

Test Accuracy: 0.6390
Test Precision: 0.6335
Test Recall: 0.6390
Test F1 Score: 0.6308
Test Confusion Matrix:
[[  1   5   7]
 [  1  11  29]
 [  1  31 119]]

Running Random Forest model...
Best Params: {'n_estimators': 150, 'min_samples_split': 5, 'max_depth': 30}
Random Forest Model Metrics:
Train Accuracy: 0.9538
Train Precision: 0.9552
Train Recall: 0.9538
Train F1 Score: 0.9511
Train Confusion Matrix:
[[ 20   4   8]
 [  0 123   9]
 [  0   1 311]]

Test Accuracy: 0.6488
Test Precision: 0.6027
Test Recall: 0.6488
Test F1 Score: 0.6182
Test Confusion Matrix:
[[  1   3   9]
 

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'sgd', 'hidden_layer_sizes': (50,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.5987
Train Precision: 0.5526
Train Recall: 0.5987
Train F1 Score: 0.5746
Train Confusion Matrix:
[[  0  14  18]
 [  0  46  86]
 [  0  73 239]]

Test Accuracy: 0.6244
Test Precision: 0.6063
Test Recall: 0.6244
Test F1 Score: 0.6144
Test Confusion Matrix:
[[  0   4   9]
 [  0  13  28]
 [  0  36 115]]

Running XGBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'subsample': 1.0, 'n_estimators': 50, 'learning_rate': 0.1}
XGBoost Model Metrics:
Train Accuracy: 0.8361
Train Precision: 0.8571
Train Recall: 0.8361
Train F1 Score: 0.8206
Train Confusion Matrix:
[[ 16   3  13]
 [  0  72  60]
 [  0   2 310]]

Test Accuracy: 0.6683
Test Precision: 0.6102
Test Recall: 0.6683
Test F1 Score: 0.6336
Test Confusion Matrix:
[[  1   2  10]
 [  2   6  33]
 [  3  18 130]]

Running AdaBoost model...


...

Best Params: {'learning_rate': 0.3, 'iterations': 150, 'depth': 6}
CatBoost Model Metrics:
Train Accuracy: 0.8950
Train Precision: 0.9028
Train Recall: 0.8950
Train F1 Score: 0.8880
Train Confusion Matrix:
[[ 16   4  12]
 [  0 100  32]
 [  0   2 310]]

Test Accuracy: 0.6341
Test Precision: 0.6050
Test Recall: 0.6341
Test F1 Score: 0.6171
Test Confusion Matrix:
[[  1   2  10]
 [  2   8  31]
 [  2  28 121]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.01}
Logistic Regression Model Metrics:
Train Accuracy: 0.6366
Train Precision: 0.5339
Train Recall: 0.6366
Train F1 Score: 0.5569
Train Confusion Matrix:
[[  0   5  27]
 [  0  16 116]
 [  0  25 287]]

Test Accuracy: 0.7220
Test Precision: 0.6229
Test Recall: 0.7220
Test F1 Score: 0.6523
Test Confusion Matrix:
[[  0   1  12]
 [  0   5  36]
 [  0   8 143]]

Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.6618
Train Precision: 0.6186
Train Recall: 0.6618
Train F1 Score: 0.5374
Train Confusion Matrix:
[[  0   1  31]
 [  0   4 128]
 [  0   1 311]]

Test Accuracy: 0.7317
Test Precision: 0.6100
Test Recall: 0.7317
Test F1 Score: 0.6309
Test Confusion Matrix:
[[  0   0  13]
 [  0   1  40]
 [  0   2 149]]

